In [2]:
import pandas as pd

In [4]:
lap_times = pd.read_csv('../dataset/lap_times.csv')
pit_stops = pd.read_csv('../dataset/pit_stops.csv')[['raceId', 'driverId', 'lap']]
races = pd.read_csv('../dataset/races.csv')[['raceId', 'circuitId']]

In [5]:
#Data Preprocessing

lap_times = lap_times.merge(races, on='raceId', how='left')
lap_times['normalized_time'] = lap_times['milliseconds'] / lap_times.groupby('raceId')['milliseconds'].transform('min')
lap_times = lap_times.merge(pit_stops[['raceId', 'driverId', 'lap']], on=['raceId', 'driverId', 'lap'], how='left', indicator=True)
lap_times['is_pit_stop'] = lap_times['_merge'] == 'both'
lap_times.drop(columns=['_merge'], inplace=True)
lap_times['is_pit_exit'] = lap_times['is_pit_stop'].shift(1, fill_value=False)
# Example: Filter out laps with normalized_time > 1.5 (assuming it's an anomaly)
lap_times = lap_times[lap_times['normalized_time'] <= 1.5]
lap_times.head()


,raceId,driverId,lap,position,time,milliseconds,circuitId,normalized_time,is_pit_stop,is_pit_exit
0,841,20,1,1,1:38.109,98109,1,1.103005,False,False
1,841,20,2,1,1:33.006,93006,1,1.045634,False,False
2,841,20,3,1,1:32.713,92713,1,1.042340,False,False
3,841,20,4,1,1:32.803,92803,1,1.043352,False,False
4,841,20,5,1,1:32.342,92342,1,1.038169,False,False


In [17]:
#Feature Engineering

features = ['circuitId', 'lap', 'is_pit_stop', 'is_pit_exit']

lap_times['lag_1'] = lap_times.groupby(['raceId', 'driverId'])['normalized_time'].shift(1)
lap_times['lag_2'] = lap_times.groupby(['raceId', 'driverId'])['normalized_time'].shift(2)
features += ['lag_1', 'lag_2']
lap_times.head()

,raceId,driverId,lap,position,time,milliseconds,circuitId,normalized_time,is_pit_stop,is_pit_exit,lag_1,lag_2
2,841,20,3,1,1:32.713,92713,1,1.042340,False,False,NaN,NaN
3,841,20,4,1,1:32.803,92803,1,1.043352,False,False,1.042340,NaN
4,841,20,5,1,1:32.342,92342,1,1.038169,False,False,1.043352,1.042340
5,841,20,6,1,1:32.605,92605,1,1.041126,False,False,1.038169,1.043352
6,841,20,7,1,1:32.502,92502,1,1.039968,False,False,1.041126,1.038169


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Drop NaNs and prepare data
lap_times.dropna(subset=features + ['normalized_time'], inplace=True)
X = lap_times[features]
y = lap_times['normalized_time']

# Split data and train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('MSE:', mean_squared_error(y_test, y_pred))


MSE: 0.0029837267659994344


In [9]:
from sklearn.ensemble import RandomForestRegressor

# Train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict using the Random Forest model
y_pred_rf = rf_model.predict(X_test)

# Calculate the MSE for the Random Forest model
mse_rf = mean_squared_error(y_test, y_pred_rf)
print('Random Forest MSE:', mse_rf)

# For comparison, print the MSE of the linear regression model if you haven't already
y_pred_lr = model.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
print('Linear Regression MSE:', mse_lr)

# Optionally, you could calculate the improvement in MSE
improvement = mse_lr - mse_rf
print('MSE Improvement:', improvement)


Random Forest MSE: 0.0017692992063879642
Linear Regression MSE: 0.0029837267659994344
MSE Improvement: 0.0012144275596114701


In [10]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
mse_gb = mean_squared_error(y_test, y_pred_gb)
print('Gradient Boosting MSE:', mse_gb)


Gradient Boosting MSE: 0.0022758533155337246


In [11]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
print('XGBoost MSE:', mse_xgb)


XGBoost MSE: 0.0018245409018525643


In [12]:
import lightgbm as lgb

lgb_model = lgb.LGBMRegressor(n_estimators=100, random_state=42)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)
mse_lgb = mean_squared_error(y_test, y_pred_lgb)
print('LightGBM MSE:', mse_lgb)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002305 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 644
[LightGBM] [Info] Number of data points in the train set: 429402, number of used features: 6
[LightGBM] [Info] Start training from score 1.062245
LightGBM MSE: 0.001955704860118339


In [13]:
from sklearn.neural_network import MLPRegressor

nn_model = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
nn_model.fit(X_train, y_train)
y_pred_nn = nn_model.predict(X_test)
mse_nn = mean_squared_error(y_test, y_pred_nn)
print('Neural Network MSE:', mse_nn)


Neural Network MSE: 0.0032711401189357073
